# 2D DC Resistivity Inversion

In [1]:
from utils.DCResistivity import DCRInversionApp
dc_app = DCRInversionApp()

# Purpose

We load observed DC data, and invert to recover a resistivity model. By altering inversion parameters and exploring correspoding inversion results, we investigate important aspects of the 2D DC resistivity inversion. 

## Outline
This notebook includes four steps:
- Step1: Load observed data
- Step2: Plot observed data
- Step3: Set mesh
- Step4: Set uncertainty
- Step5: Run inversion
- Step6: Explore inversion results
- Step7: Run inversion to compute DOI index
- Step8: Plot DOI index

## Step 1: Load observed data

- `filename`: file name for the observed data
- `topography`: file name for the topography data
- `load`: will load the observation file if this is checked
- `input type`: type of the input file

In [2]:
dc_app.interact_load_obs()

interactive(children=(Dropdown(description='filename: ', index=8, layout=Layout(width='max-content'), options=…

## Step 2: Plot observed data

Once loaded, we examine the data. From the *pseudo-section* plot, we can determine the background resistivity of the Earth and highlight any interesting features. To view the data, we can use the following parameters:

- `data type`: type of the data
- `plot type`: type of the data plot
- `aspect ratio`: aspect ratio for pseudo-section plot. Changes size of plot

In [12]:
dc_app.interact_plot_obs_data()

interactive(children=(ToggleButtons(description='data type', options=('apparent_resistivity', 'volt'), value='…

## Step 3: Set a 2D tensor mesh 

Designing an mesh is very important. If we use smaller cells in our mesh, we solve the problem more accurately. However, we cannot make the cells too small because it requires too much computer memory. Parameters for designing a mesh are:

- `dx`: horizontal cell size
- `dz`: vertical cell size
- `corezlength`: the depth that the core mesh extends
- `xpad`: thickness of padding region in the horizontal direction
- `zpad`: thickness of padding region in the vertical direction
- `mesh_type`: tensor or tree mesh

Some things to consider when designing a mesh:

- Using a 'Tree Mesh' will **always** reduce the number of cells.


- To choose a good *dx* value, think about the minimum electrode spacing. The value of *dx* should be 25%-40% as large as the minimum electrode spacing. So if minimum electrode spacing is 10 m, you should choose a *dx* between 2.5 m and 4 m.


- It is common to let *dz* be the same as *dx*. Sometimes we set *dz* to be half the size of *dx*.


- *corezlength* defines a depth limit for where you think most currents are. Below this depth, you are assuming the currents are very small. A good first guess would be to set *corezlength* to be equal to the maximum AB electrode spacing.


- The padding (*xpad* and *zpad*) needs to be big enough so that the currents are zero on the edges of your mesh. For a Wenner-Schlumberger survey, set the padding region to be 1-2 times larger than the maximum AB spacing. For a dipole-dipole survey, set the padding region to be 2-3 times larger than the maximum AB spacing

In [13]:
dc_app.interact_set_mesh()

>> suggested dx: 2.5 m
>> suggested dz: 1.25 m
>> suggested x padding: 393.3333333333333 m
>> suggested z padding: 393.3333333333333 m
>> suggested corezlength: 196.66666666666666 m


interactive(children=(FloatText(value=2.5, description='dx'), FloatText(value=1.25, description='dz'), FloatTe…

## Step 4: Set uncertainty

The uncertainties are an estimate of the level of noise on your data. If your data have more noise (error bars are larger), you must assign larger uncertainties. The equation for the uncertainty you assign to each data value is:

$$ \text{uncertainty} = 0.01 \times \text{percentage}\times|d^{obs}| + \text{floor}$$

- **percentage (%):** percentage uncertainty
- **floor (V):** floor uncertainty. The raw data are volts, not apparent resistivities.

For apparent resistivity data, it is common to choose a percentage between 2%-10%. The floor value is usually something very small; less than 0.0001 $\Omega m$

In [14]:
dc_app.interact_set_uncertainty()

interactive(children=(FloatText(value=5.0, description='percentage'), FloatText(value=0.0, description='floor'…

## Step 5: Run inversion

Here, we define parameters necessary to complete the inversion. These parameters are:

- $\rho_0$: initial resistivity model
- $\rho_{ref}$: reference resistivity model
- $\alpha_s$: controls how much the inversion wants to recover a model similar to the reference model
- $\alpha_x$: controls how much the inversion wants to recover a model that is horizontally smooth
- $\alpha_z$: controls how much the inversion wants to recover a model that is vertically smooth
- `maxIter`: maximum number of iteration (10-15 is ideal)
- `chifact`: chifactor for the target misfit
- `beta0_ratio`: ratio to set the initial beta (default value is 10)
- `coolingFactor`: cooling factor to cool beta (default value is 2)
- `n_iter_per_beta`: # of interation for each beta value 
- `run`: run inversion if this is checked

Here are some things to consider when choosing these parameters:

- If you make $\alpha_s$ much larger than $\alpha_x$ and $\alpha_z$, the inversion will try very hard to give you an answer that is similar to the reference model. If you make $\alpha_x$ and $\alpha_z$ much larger than $\alpha_s$, the inversion will try very hard to find a smooth model that explains the data. To start, it is good to set $\alpha_s = 0.01$, $\alpha_x = 1$ and $\alpha_z = 1$.
- If you set *coolingFactor* to a number equal or larger than 4, you should set the *n_iter_per_beta* to a value of 2. This makes sure you solve the problem accurately.
- If the inversion only requires a few iterations, your uncertainties may be too large.
- If your inversion does not reach *target misfit*, your uncertainties may be too small. The number of layers and their thicknessess may also be incorrect.

In [15]:
dc_app.interact_run_inversion()

interactive(children=(FloatText(value=504.0, description='$\\rho_0$'), FloatText(value=504.0, description='$\\…

## Step 6: Explore inversion results

- `iteration`: inversion iteration
- `curve type`:type of the curve (this is active when `plot type`=`misfit_curve`)
- `scale`: linear or log scale (this is active when `plot type`=`misfit_curve`)
- `plot type`: type of the plot
- `show grid?`: show mesh grid when this is checked
- `show core?`: show only core domain when this is checked. this is better to interpret results.
- `aspect ratio`: aspect ratio for data misfit plot

In [7]:
dc_app.interact_plot_inversion_results()

>> no inversion results yet


## Step 7: Run inversion to compute DOI index

Depth of investigation (DOI) index can be computed by following equation (Oldenburg and Li, 1999):

$$ \text{doi index} = \frac{m^1-m^2}{m_{ref}^1-m_{ref}^2}$$

where 

- $m^1$: inversion model 1 (from Step5)
- $m^2$: inversion model 2 (same inversion parameters with model 1 except for `m0` and `mref`
- $m_{ref}^1$: reference model 1 (used for Step 5)
- $m_{ref}^2$: reference model 2 (=$m_{ref}^1 \times$ factor)

Here a constant factor is multiplied to generate a new reference model. 
Below app will run inversion to obtain another inversion model ($m^2$), which will allow us to 
compute DOI index in the following app. 

### Parameters
- `factor`: constant factor to compute a new reference model
- `run`: if checked, then a new inverion will run

In [8]:
dc_app.interact_run_doi()

interactive(children=(FloatText(value=0.5, description='factor'), Checkbox(value=False, description='run'), Ou…

##  Step 8: Plot DOI index


- `plot type`: type of the plot
- `doi_level`: level of the doi index
- `scale`: linear or log scale (this is active when `plot type`=`models` or `final`)
- `show grid?`: show mesh grid when this is checked
- `show core?`: show only core domain when this is checked
- `aspect_ratio`: vertical to horizontal ratio

In [9]:
dc_app.interact_plot_doi_results()

>> an inversion for doi calculation is needed to be run
